In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.interpolate import make_interp_spline
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
# Daten einlesen
df = pd.read_csv('reden_relevant.csv')
print(len(df))

In [ ]:
reden_pro_partei = df['partei'].value_counts()

# Ausgabe der Anzahl der Reden je Partei
for party, count in reden_pro_partei.items():
    print(f"{party}: {count} Reden")

In [ ]:
# Ersetzen von PDS mit der Linken
df['partei'] = df['partei'].replace({'PDS': 'LINKE'})

In [ ]:
# Funktion zur Extraktion der Terme mit Häufigkeiten
def extract_terms_with_frequencies(text, term_list, ngram_range=(1, 2)):
    vectorizer = CountVectorizer(ngram_range=ngram_range, vocabulary=term_list, token_pattern=r'\b\w+\b')
    term_counts = vectorizer.fit_transform([text]).toarray().flatten()
    term_freq = {term: count for term, count in zip(term_list, term_counts) if count > 0}
    return term_freq

## Sentiment für einzelne Parteien

### LNTWS

LNTWS-Sentiment berechnen.

In [ ]:
terms_df = pd.read_csv('lntws_wörterbuch.csv', delimiter=';')

expansionary_terms = terms_df['expansive_terme'].dropna().tolist()
contractionary_terms = terms_df['restriktive_terme'].dropna().tolist()
expansionary_terms = [term.replace("_", " ") for term in expansionary_terms]
contractionary_terms = [term.replace("_", " ") for term in contractionary_terms]

In [ ]:
# Konvertiere die Spalte "Datum" in datetime
df['Datum'] = pd.to_datetime(df['Datum'], errors='coerce')  # Handle errors by setting invalid dates to NaT
df.dropna(subset=['Datum'], inplace=True)  # Remove rows with invalid dates

# Liste zur Speicherung der Ergebnisse
results = []

for index, row in df.iterrows():
    text = row['text_preprocessed_specific']
    date = row['Datum']
    party = row['partei']  # Partei des Redners
    year = date.year
    quarter = (date.month - 1) // 3 + 1  # Bestimme Quartal
    month = date.month
    total_words = row['länge_bereinigt_spezifisch']

    # Extrahiere expansive und restriktive Terme
    expansive_terms_freq = extract_terms_with_frequencies(text, expansionary_terms, ngram_range=(1, 2))
    contractionary_terms_freq = extract_terms_with_frequencies(text, contractionary_terms, ngram_range=(1, 2))

    # Berechne die Summe der expansiven und restriktiven Terme
    expansive_count = sum(expansive_terms_freq.values())
    contractionary_count = sum(contractionary_terms_freq.values())

    # Berechne das Sentiment
    sentiment_score = (expansive_count - contractionary_count) / total_words if total_words > 0 else 0

    # Füge die Ergebnisse hinzu
    results.append({
        'Datum': date,
        'Jahr': year,
        'Quartal': quarter,
        'Monat': month,
        'Partei': party,
        'Sentiment Score': sentiment_score
    })

# Konvertiere die Ergebnisse in einen DataFrame
results_df = pd.DataFrame(results)

# Beispielhafte Ausgabe zur Kontrolle
print(results_df.head())

### BPWS

BPWS-Sentiment berechnen.

In [ ]:
# Lade die verarbeiteten Wörter
terms_df = pd.read_csv('bpws_wörterbuch.csv', delimiter=';')

positives = terms_df['lemma_positiv'].dropna().tolist()
negatives = terms_df['lemma_negativ'].dropna().tolist()

In [ ]:
# Konvertieren der Spalte "Datum" in datetime
df['Datum'] = pd.to_datetime(df['Datum'], errors='coerce')  # Handle errors by setting invalid dates to NaT
df.dropna(subset=['Datum'], inplace=True)  # Remove rows with invalid dates

# Liste zur Speicherung der Ergebnisse
results = []

for index, row in df.iterrows():
    text = row['text_preprocessed_specific']
    date = row['Datum']
    party = row['partei']  # Partei des Redners
    year = date.year
    quarter = (date.month - 1) // 3 + 1  # Bestimme Quartal
    month = date.month
    total_words = row['länge_bereinigt_spezifisch']

    # Extrahieren positiver und negativer Terme
    negatives_freq = extract_terms_with_frequencies(text, negatives)
    positives_freq = extract_terms_with_frequencies(text, positives)

    # Berechnen der Summe der positiven und negativen Terme
    negative_count = sum(negatives_freq.values())
    positive_count = sum(positives_freq.values())

    # Berechnen des Sentiments
    sentiment_score = (positive_count - negative_count) / total_words if total_words > 0 else 0

    # Hinzufügen der Ergebnisse
    results.append({
        'Datum': date,
        'Jahr': year,
        'Quartal': quarter,
        'Monat': month,
        'Partei': party,
        'Sentiment Score': sentiment_score
    })

# Konvertieren der Ergebnisse in einen DataFrame
results_df = pd.DataFrame(results)

# Beispielhafte Ausgabe zur Kontrolle
print(results_df.head())

## Graphischer Vergleich des Sentiments in Artikel und Reden
Dafür werden die Reden nach den einzelnen Parteien aufgeteilt und pro Jahr aggregiert.

In [ ]:
# Aggregieren der Ergebnisse nach Partei und Quartal
sentiment_quarterly = results_df.groupby(['Jahr', 'Quartal', 'Partei'])['Sentiment Score'].mean().reset_index()

# Aggregieren der Ergebnisse nach Partei und Monat
sentiment_monthly = results_df.groupby(['Jahr', 'Monat', 'Partei'])['Sentiment Score'].mean().reset_index()

In [ ]:
# Speichern der Ergebnisse als CSV-Dateien
sentiment_quarterly.to_csv('sentiment_partei_bpw_quartal.csv', index=False)
sentiment_monthly.to_csv('sentiment_partei_bpw_monat.csv', index=False)

Erstellen der Graphik zum visuellen Vergleich der Entwicklung des Sentiments der einzelnen Parteien. (Nur BPWS-Sentiment)

In [ ]:
# Definieren der Ereignisse und deren Zeiträume in datetime
events = {
    "Dotcom-Blase": ("2001-02-01", "2003-06-30"),
    "Weltwirtschaftskrise": ("2008-01-01", "2009-04-30"),
    "Coronakrise": ("2019-09-01", "2020-04-30")
}

Monatlich

In [ ]:
# Konvertieren der Monate in datetime-Format
sentiment_monthly['Datum'] = pd.to_datetime(
    sentiment_monthly['Jahr'].astype(int).astype(str) + '-' + sentiment_monthly['Monat'].astype(str).str.zfill(2) + '-01'
)

# Begrenzen der Daten auf den Zeitraum bis Ende Q3 2021
end_date = pd.to_datetime("2021-09-30")
sentiment_monthly = sentiment_monthly[sentiment_monthly['Datum'] <= end_date]

# Erstellen des Plots
plt.figure(figsize=(12, 6))

# Farben für jede Partei definieren
farben = {
    'CDU/CSU': 'black',
    'SPD': 'red',
    'FDP': 'gold',
    'LINKE': 'deeppink',
    'GRÜNE': 'limegreen',
    'AfD': 'dodgerblue',
    'Cabinet': 'midnightblue'
}

# Plot für monatliches Sentiment jeder Partei
# Plot für monatliches Sentiment jeder Partei
for party in sentiment_monthly['Partei'].unique():
    if party in farben:  # Nur Parteien plotten, die in `farben` definiert sind
        party_data = sentiment_monthly[sentiment_monthly['Partei'] == party]
        plt.plot(party_data['Datum'], party_data['Sentiment Score'], label=f'{party}', color=farben[party])

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=sentiment_monthly['Datum'].min(),
                           end=sentiment_monthly['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
# plt.title('Monatliche Sentiment-Entwicklung der Parteien (BPWS)', fontsize=14)
plt.ylabel('Sentiment-Score', fontsize=12)
plt.grid(alpha=0.3)

# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=7, frameon=False)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('reden_sentiment_parteien_bpw_relevant_m.png', format='png', dpi=600)

# Zeige den Plot
plt.show()

Quartalsweise

In [ ]:
# Konvertieren der Quartale in datetime-Format
sentiment_quarterly['Datum'] = pd.PeriodIndex(
    sentiment_quarterly['Jahr'].astype(int).astype(str) + 'Q' + sentiment_quarterly['Quartal'].astype(int).astype(str), freq='Q'
).to_timestamp()

# Begrenzen der Daten auf den Zeitraum bis Ende Q3 2021
end_date = pd.to_datetime("2021-09-30")
sentiment_quarterly = sentiment_quarterly[sentiment_quarterly['Datum'] <= end_date]

# Erstellen des Plots
plt.figure(figsize=(12, 6))

for party in sentiment_quarterly['Partei'].unique():
    if party in party_colors:  # Nur Parteien plotten, die in `party_colors` definiert sind
        party_data = sentiment_quarterly[sentiment_quarterly['Partei'] == party]
        plt.plot(party_data['Datum'], party_data['Sentiment Score'], label=f'{party}', color=party_colors[party])

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=sentiment_quarterly['Datum'].min(),
                           end=sentiment_quarterly['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
# plt.title('Quartalsweise Entwicklung des Sentiments pro Partei', fontsize=14)
plt.ylabel('Sentiment-Score', fontsize=12)
plt.grid(alpha=0.3)

# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=7, frameon=False)
plt.tight_layout()

# Speichern der Artikel
plt.savefig('reden_sentiment_parteien_bpw_relevant_q.png', format='png', dpi=600)

plt.show()

## Berechnen der Korrelation

### Korrelation zwischen den einzelnen Parteien

In [ ]:
import seaborn as sns

In [ ]:
# Filtern der Parteien auf die, die in farben enthalten sind
filtered_parties = [party for party in sentiment_quarterly['Partei'].unique() if party in farben.keys()]

# Initialisieren eines leeren DataFrames für die Ergebnisse
correlation_matrix = pd.DataFrame(index=filtered_parties, columns=filtered_parties)

# Iterieren über alle gefilterten Parteien-Paare, um die Korrelationen zu berechnen
for party1 in filtered_parties:
    for party2 in filtered_parties:
        if party1 == party2:
            # Setzen der diagonalen Werte auf 1 (Selbstkorrelation)
            correlation_matrix.loc[party1, party2] = 1.0
        else:
            # Filtern der Daten für beide Parteien
            data1 = sentiment_quarterly[sentiment_quarterly['Partei'] == party1][['Datum', 'Sentiment Score']]
            data2 = sentiment_quarterly[sentiment_quarterly['Partei'] == party2][['Datum', 'Sentiment Score']]

            # Zusammenführen der Daten anhand des Datums (nur gemeinsame Zeiträume bleiben erhalten)
            merged_data = pd.merge(data1, data2, on='Datum', suffixes=(f'_{party1}', f'_{party2}'))

            # Berechnen der Korrelation zwischen den Sentimentwerten
            if not merged_data.empty:
                correlation = merged_data[f'Sentiment Score_{party1}'].corr(merged_data[f'Sentiment Score_{party2}'])
                correlation_matrix.loc[party1, party2] = correlation
            else:
                # Wenn keine gemeinsamen Datenpunkte vorhanden sind, setze NaN
                correlation_matrix.loc[party1, party2] = np.nan

# Konvertieren der Korrelationsmatrix in numerische Werte
correlation_matrix = correlation_matrix.astype(float)

# Ausgabe der Korrelationsmatrix
print("Korrelationen zwischen den Parteien (gefiltert):")
print(correlation_matrix)

# Visualisieren der Korrelationsmatrix als Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".3f", cbar=True,
            xticklabels=filtered_parties, yticklabels=filtered_parties)
plt.title('Korrelationen der Sentiment-Entwicklung zwischen den Parteien', fontsize=14)
plt.tight_layout()
plt.savefig('korrelationen_parteien_bpw_relevant_q.png', dpi=600)
plt.show()

P-Werte

In [ ]:
from scipy.stats import pearsonr

In [ ]:
# Initialisieren des DataFrames für Korrelationen und P-Werte
correlation_matrix = pd.DataFrame(index=filtered_parties, columns=filtered_parties)
p_value_matrix = pd.DataFrame(index=filtered_parties, columns=filtered_parties)

# Iterieren über alle gefilterten Parteien-Paare
for party1 in filtered_parties:
    for party2 in filtered_parties:
        if party1 == party2:
            # Setzen der diagonalen Werte (Selbstkorrelation) auf 1
            correlation_matrix.loc[party1, party2] = 1.0
            p_value_matrix.loc[party1, party2] = 0.0  # Kein P-Wert für Selbstkorrelation
        else:
            # Filtern der Daten für beide Parteien
            data1 = sentiment_quarterly[sentiment_quarterly['Partei'] == party1][['Datum', 'Sentiment Score']]
            data2 = sentiment_quarterly[sentiment_quarterly['Partei'] == party2][['Datum', 'Sentiment Score']]

            # Zusammenführen der Daten anhand des Datums (nur gemeinsame Zeiträume bleiben erhalten)
            merged_data = pd.merge(data1, data2, on='Datum', suffixes=(f'_{party1}', f'_{party2}'))

            # Berechnen der Korrelation und der P-Werte
            if not merged_data.empty:
                corr, p_val = pearsonr(
                    merged_data[f'Sentiment Score_{party1}'],
                    merged_data[f'Sentiment Score_{party2}']
                )
                correlation_matrix.loc[party1, party2] = corr
                p_value_matrix.loc[party1, party2] = p_val
            else:
                # Wenn keine gemeinsamen Datenpunkte vorhanden sind, setze NaN
                correlation_matrix.loc[party1, party2] = np.nan
                p_value_matrix.loc[party1, party2] = np.nan

# Konvertieren zu float für numerische Werte
correlation_matrix = correlation_matrix.astype(float)
p_value_matrix = p_value_matrix.astype(float)

# Ausgabe der Ergebnisse
# print("Korrelationsmatrix:")
# print(correlation_matrix)

print("\nP-Wert-Matrix:")
print(p_value_matrix)

Visualisieren in einer Heat-Map

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(p_value_matrix, annot=True, fmt=".2g", cmap="viridis", cbar=True)
plt.title("P-Wert-Matrix der Sentiments zwischen den Parteien")
plt.show()

Monatliche Korrelation

In [ ]:
# Initialisieren der DataFrames für Korrelationen und P-Werte
correlation_matrix_monthly = pd.DataFrame(index=filtered_parties, columns=filtered_parties)
p_value_matrix_monthly = pd.DataFrame(index=filtered_parties, columns=filtered_parties)

# Iterieren über alle gefilterten Parteien-Paare
for party1 in filtered_parties:
    for party2 in filtered_parties:
        if party1 == party2:
            # Setze die diagonalen Werte (Selbstkorrelation) auf 1
            correlation_matrix_monthly.loc[party1, party2] = 1.0
            p_value_matrix_monthly.loc[party1, party2] = 0.0  # Kein P-Wert für Selbstkorrelation
        else:
            # Filtern der Daten für beide Parteien
            data1 = sentiment_monthly[sentiment_monthly['Partei'] == party1][['Datum', 'Sentiment Score']]
            data2 = sentiment_monthly[sentiment_monthly['Partei'] == party2][['Datum', 'Sentiment Score']]

            # Zusammenführen der Daten anhand des Datums (nur gemeinsame Zeiträume bleiben erhalten)
            merged_data = pd.merge(data1, data2, on='Datum', suffixes=(f'_{party1}', f'_{party2}'))

            # Berechnen der Korrelation und der P-Werte
            if not merged_data.empty:
                corr, p_val = pearsonr(
                    merged_data[f'Sentiment Score_{party1}'],
                    merged_data[f'Sentiment Score_{party2}']
                )
                correlation_matrix_monthly.loc[party1, party2] = corr
                p_value_matrix_monthly.loc[party1, party2] = p_val
            else:
                # Wenn keine gemeinsamen Datenpunkte vorhanden sind, setze NaN
                correlation_matrix_monthly.loc[party1, party2] = np.nan
                p_value_matrix_monthly.loc[party1, party2] = np.nan

# Konvertieren zu float für numerische Werte
correlation_matrix_monthly = correlation_matrix_monthly.astype(float)
p_value_matrix_monthly = p_value_matrix_monthly.astype(float)

# Ausgabe der Ergebnisse
print("Monatliche Korrelationsmatrix:")
print(correlation_matrix_monthly)

print("\nMonatliche P-Wert-Matrix:")
print(p_value_matrix_monthly)

Visualisieren der Korrelation

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_monthly, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Korrelationsmatrix der Sentiments (monatlich)")
plt.tight_layout()
plt.savefig('korrelationen_parteien_bpw_relevant_m.png', dpi=600)
plt.show()

Visualisieren der P-Werte

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(p_value_matrix_monthly, annot=True, fmt=".2g", cmap="viridis", cbar=True)
plt.title("P-Wert-Matrix der Sentiments (monatlich)")
plt.show()

### Korrelation von Artikeln und Parteireden

In [ ]:
df_am = pd.read_csv('artikel_bpw_relevant_monat.csv')
df_aq = pd.read_csv('artikel_bpw_relevant_quartal.csv')

Mit dem BPWS-Sentiment.

In [ ]:
df_am.rename(columns={'BPW Score': 'Sentiment Score'}, inplace=True)
df_aq.rename(columns={'BPW Score': 'Sentiment Score'}, inplace=True)

In [ ]:
df_am.head()

#### Monatlich

In [ ]:
# Laden der Artikel-Sentimentdaten
df_am['Datum'] = pd.to_datetime(df_am['Jahr'].astype(str) + '-' + df_am['Monat'].astype(str).str.zfill(2) + '-01')

# Initialisieren der DataFrames für Korrelationen und P-Werte
correlation_matrix_monthly = pd.DataFrame(index=filtered_parties + ['Artikel'], columns=filtered_parties + ['Artikel'])
p_value_matrix_monthly = pd.DataFrame(index=filtered_parties + ['Artikel'], columns=filtered_parties + ['Artikel'])

# Iterieren über alle Parteien und füge "Artikel" hinzu
all_entities = filtered_parties + ['Artikel']

for entity1 in all_entities:
    for entity2 in all_entities:
        if entity1 == entity2:
            # Setze die diagonalen Werte (Selbstkorrelation) auf 1
            correlation_matrix_monthly.loc[entity1, entity2] = 1.0
            p_value_matrix_monthly.loc[entity1, entity2] = 0.0  # Kein P-Wert für Selbstkorrelation
        else:
            # Filtern der Daten
            if entity1 == 'Artikel':
                data1 = df_am[['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity1}'})
            else:
                data1 = sentiment_monthly[sentiment_monthly['Partei'] == entity1][['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity1}'})

            if entity2 == 'Artikel':
                data2 = df_am[['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity2}'})
            else:
                data2 = sentiment_monthly[sentiment_monthly['Partei'] == entity2][['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity2}'})

            # Zusammenführen der Daten anhand des Datums (nur gemeinsame Zeiträume bleiben erhalten)
            merged_data = pd.merge(data1, data2, on='Datum')

            # Berechne Korrelation und P-Wert
            if not merged_data.empty:
                corr, p_val = pearsonr(
                    merged_data[f'Sentiment Score_{entity1}'],
                    merged_data[f'Sentiment Score_{entity2}']
                )
                correlation_matrix_monthly.loc[entity1, entity2] = corr
                p_value_matrix_monthly.loc[entity1, entity2] = p_val
            else:
                # Wenn keine gemeinsamen Datenpunkte vorhanden sind, setze NaN
                correlation_matrix_monthly.loc[entity1, entity2] = np.nan
                p_value_matrix_monthly.loc[entity1, entity2] = np.nan

# Konvertieren zu float für numerische Werte
correlation_matrix_monthly = correlation_matrix_monthly.astype(float)
p_value_matrix_monthly = p_value_matrix_monthly.astype(float)


Visualisieren

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_monthly, annot=True, fmt=".3f", cmap="coolwarm", cbar=True)
plt.title("Korrelationsmatrix der Sentiments (monatlich)")
plt.savefig('korrelationen_parteien_artikel_bpw_relevant_m.png', dpi=600)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(p_value_matrix_monthly, annot=True, fmt=".3g", cmap="viridis", cbar=True)
plt.title("P-Wert-Matrix der Sentiments (monatlich)")
plt.show()

#### Quartalsweise

In [ ]:
# Laden der Artikel-Sentimentdaten
df_aq['Datum'] = pd.PeriodIndex(df_aq['Jahr'].astype(str) + 'Q' + df_aq['Quartal'].astype(str), freq='Q').to_timestamp()

# Initialisiere die DataFrames für Korrelationen und P-Werte
correlation_matrix_quarterly = pd.DataFrame(index=filtered_parties + ['Artikel'], columns=filtered_parties + ['Artikel'])
p_value_matrix_quarterly = pd.DataFrame(index=filtered_parties + ['Artikel'], columns=filtered_parties + ['Artikel'])

# Iteriere über alle Parteien und füge "Artikel" hinzu
for entity1 in all_entities:
    for entity2 in all_entities:
        if entity1 == entity2:
            # Setze die diagonalen Werte (Selbstkorrelation) auf 1
            correlation_matrix_quarterly.loc[entity1, entity2] = 1.0
            p_value_matrix_quarterly.loc[entity1, entity2] = 0.0  # Kein P-Wert für Selbstkorrelation
        else:
            # Filtern der Daten
            if entity1 == 'Artikel':
                data1 = df_aq[['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity1}'})
            else:
                data1 = sentiment_quarterly[sentiment_quarterly['Partei'] == entity1][['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity1}'})

            if entity2 == 'Artikel':
                data2 = df_aq[['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity2}'})
            else:
                data2 = sentiment_quarterly[sentiment_quarterly['Partei'] == entity2][['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': f'Sentiment Score_{entity2}'})

            # Zusammenführen der Daten anhand des Datums (nur gemeinsame Zeiträume bleiben erhalten)
            merged_data = pd.merge(data1, data2, on='Datum')

            # Berechne Korrelation und P-Wert
            if not merged_data.empty:
                corr, p_val = pearsonr(
                    merged_data[f'Sentiment Score_{entity1}'],
                    merged_data[f'Sentiment Score_{entity2}']
                )
                correlation_matrix_quarterly.loc[entity1, entity2] = corr
                p_value_matrix_quarterly.loc[entity1, entity2] = p_val
            else:
                # Wenn keine gemeinsamen Datenpunkte vorhanden sind, setze NaN
                correlation_matrix_quarterly.loc[entity1, entity2] = np.nan
                p_value_matrix_quarterly.loc[entity1, entity2] = np.nan

# Konvertieren zu float für numerische Werte
correlation_matrix_quarterly = correlation_matrix_quarterly.astype(float)
p_value_matrix_quarterly = p_value_matrix_quarterly.astype(float)


Visualisieren

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_quarterly, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Korrelationsmatrix der Sentiments (quartalsweise)")
plt.savefig('korrelationen_parteien_artikel_bpw_relevant_q.png', dpi=600)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(p_value_matrix_quarterly, annot=True, fmt=".2g", cmap="viridis", cbar=True)
plt.title("P-Wert-Matrix der Sentiments (quartalsweise)")
plt.show()

### Korrelation mit allen Reden

In [ ]:
# Aggregieren der Sentimentwerte aller Reden unabhängig von der Parteizugehörigkeit
# Monatliche Aggregation
sentiment_monthly_total = sentiment_monthly.groupby(['Datum'])[['Sentiment Score']].mean().reset_index()

# Quartalsweise Aggregation
sentiment_quarterly_total = sentiment_quarterly.groupby(['Datum'])[['Sentiment Score']].mean().reset_index()

# Berechnung der Korrelation mit den Artikelsentimentwerten

# Monatsweise Korrelation
merged_monthly = pd.merge(
    sentiment_monthly_total.rename(columns={'Sentiment Score': 'Bundestag Sentiment'}),
    df_am.rename(columns={'Sentiment Score': 'Artikel Sentiment'}),
    on='Datum'
)

monthly_correlation, monthly_p_value = pearsonr(
    merged_monthly['Bundestag Sentiment'], merged_monthly['Artikel Sentiment']
)

# Quartalsweise Korrelation
merged_quarterly = pd.merge(
    sentiment_quarterly_total.rename(columns={'Sentiment Score': 'Bundestag Sentiment'}),
    df_aq.rename(columns={'Sentiment Score': 'Artikel Sentiment'}),
    on='Datum'
)

quarterly_correlation, quarterly_p_value = pearsonr(
    merged_quarterly['Bundestag Sentiment'], merged_quarterly['Artikel Sentiment']
)

# Ausgabe der Ergebnisse
print("Korrelation (monatlich):")
print(f"Korrelationskoeffizient: {monthly_correlation:.2f}")
print(f"P-Wert: {monthly_p_value:.2g}")

print("\nKorrelation (quartalsweise):")
print(f"Korrelationskoeffizient: {quarterly_correlation:.2f}")
print(f"P-Wert: {quarterly_p_value:.2g}")